In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path

import talib

In [2]:
class CSVDataHandler:
    def __init__(self, csv_dir, symbol_list, timeframe):
        self.csv_dir = csv_dir
        self.symbol_list = symbol_list
        self.timeframe = timeframe
        
        self.symbol_data = dict()
        self.latest_symbol_data = dict()
        
        self.continue_backtest = True
        
        self._import_symbol_data()
        
    def _import_symbol_data(self):
        csv_files_path = f'{Path().absolute()}/{self.csv_dir}'
        columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        combined_symbol_index = None
        
        for symbol in symbol_list:
            symbol_csv_path = f'{csv_files_path}/{symbol}_{self.timeframe}.csv'
            self.symbol_data[symbol] = pd.read_csv(symbol_csv_path, header=None, index_col=0, names=columns)
            
            self.symbol_data[symbol].index = pd.to_datetime(self.symbol_data[symbol].index, unit='ms')
            self.symbol_data[symbol] = self.symbol_data[symbol].drop_duplicates()
            self.symbol_data[symbol] = self.symbol_data[symbol].sort_index()
            
            if combined_symbol_index is None:
                combined_symbol_index = self.symbol_data[symbol].index
            else:
                combined_symbol_index = combined_symbol_index.union(self.symbol_data[symbol].index)
            
            self.latest_symbol_data[symbol] = list()
            
        for symbol in symbol_list:
            self.symbol_data[symbol] = self.symbol_data[symbol].reindex(index=combined_symbol_index, method='pad', fill_value=0).itertuples()
            
    def _get_new_bar(self, symbol):
        for bar in self.symbol_data[symbol]:
            yield bar
            
    def update_bars(self):
        for symbol in self.symbol_list:
            try:
                bar = next(self._get_new_bar(symbol))
            except StopIteration:
                self.continue_backtest = False
            else:
                if bar is not None:
                    self.latest_symbol_data[symbol].append(bar)
                    
    def get_latest_bar(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars(self, symbol, N=1):
        try:
            return self.latest_symbol_data[symbol][-N:]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_datetime(self, symbol):
        try:
            return self.latest_symbol_data[symbol][-1].Index
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bar_value(self, symbol, value_type):
        try:
            return getattr(self.latest_symbol_data[symbol][-1], value_type)
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
            
    def get_latest_bars_values(self, symbol, value_type, N=1):
        try:
            bars = self.latest_symbol_data[symbol][-N:]
        except KeyError:
            print("That symbol is not available in the historical data set.")
            raise
        else:
            return np.array([getattr(bar, value_type) for bar in bars])

In [3]:
symbol_list = ['BTC-USDT', 'ETH-BTC']
data_handler = CSVDataHandler('exchange_data', symbol_list, timeframe='1h')

In [4]:
print(data_handler.symbol_data)
print(data_handler.latest_symbol_data)

{'BTC-USDT': <map object at 0x118750e48>, 'ETH-BTC': <map object at 0x10c0f3470>}
{'BTC-USDT': [], 'ETH-BTC': []}


In [6]:
for i in range(608):
    data_handler.update_bars()

In [114]:
def MFI(hlcv, window=14):
    hlcv_window = hlcv[-window:]
    
    high = hlcv_window[:, 0]
    low = hlcv_window[:, 1]
    close = hlcv_window[:, 2]
    volume = hlcv_window[:, 3]
    
    typical_price = (high + low + close) / 3
    raw_money_flow = typical_price * volume
    typical_price_diff = np.diff(typical_price, prepend=np.nan)
    
    gains = raw_money_flow[typical_price_diff > 0]
    losses = raw_money_flow[typical_price_diff < 0]
    
    try:
        money_flow_ratio = gains.sum() / losses.sum()
    except ZeroDivisionError:
        money_flow_ratio = 1

    money_flow_index = 100 - (100 / (1 + money_flow_ratio))

    return money_flow_index

In [159]:
bars = np.array(data_handler.get_latest_bars('ETH-BTC', N=50))
hlc = bars[:, 2:-1]
hlc

array([[0.06299400000000001, 0.0626, 0.0629],
       [0.06304900000000001, 0.062593, 0.062885],
       [0.06303099999999999, 0.062238999999999996, 0.06232000000000001],
       [0.062373000000000005, 0.06155, 0.061975],
       [0.062282000000000004, 0.0618, 0.061996],
       [0.062282000000000004, 0.061613, 0.062166],
       [0.06217999999999999, 0.0615, 0.061567],
       [0.061764, 0.061, 0.061015999999999994],
       [0.061685000000000004, 0.061, 0.061599],
       [0.0622, 0.06146, 0.061497],
       [0.061958000000000006, 0.061336, 0.061949000000000004],
       [0.062046000000000004, 0.061642999999999996, 0.061712],
       [0.06209, 0.061538999999999996, 0.06176],
       [0.0618, 0.061360000000000005, 0.061475],
       [0.06185, 0.061364, 0.061701],
       [0.0621, 0.061554, 0.061923],
       [0.0621, 0.061834, 0.061949000000000004],
       [0.062189, 0.06186900000000001, 0.061914],
       [0.062082000000000005, 0.061742, 0.061815999999999996],
       [0.061995, 0.061161, 0.061259],
 

In [145]:
%%timeit
latest_close = data_handler.get_latest_bars_values('ETH-BTC', 'close', N=250)
talib.RSI(latest_close)[-1]

74.7 µs ± 7.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [146]:
start = bars[-1, 0].hour + 1 + 24
end = bars[-1, 0].hour + 1
prev_day_bars = bars[-start:-end]
high = prev_day_bars[:, 2].max()
low = prev_day_bars[:, 3].min()
close = prev_day_bars[:, 4][-1]

In [188]:
bars[:, [0, 2, 3, 4]]

array([[Timestamp('2018-03-22 07:00:00'), 0.06299400000000001, 0.0626,
        0.0629],
       [Timestamp('2018-03-22 08:00:00'), 0.06304900000000001, 0.062593,
        0.062885],
       [Timestamp('2018-03-22 09:00:00'), 0.06303099999999999,
        0.062238999999999996, 0.06232000000000001],
       [Timestamp('2018-03-22 10:00:00'), 0.062373000000000005, 0.06155,
        0.061975],
       [Timestamp('2018-03-22 11:00:00'), 0.062282000000000004, 0.0618,
        0.061996],
       [Timestamp('2018-03-22 12:00:00'), 0.062282000000000004, 0.061613,
        0.062166],
       [Timestamp('2018-03-22 13:00:00'), 0.06217999999999999, 0.0615,
        0.061567],
       [Timestamp('2018-03-22 14:00:00'), 0.061764, 0.061,
        0.061015999999999994],
       [Timestamp('2018-03-22 15:00:00'), 0.061685000000000004, 0.061,
        0.061599],
       [Timestamp('2018-03-22 16:00:00'), 0.0622, 0.06146, 0.061497],
       [Timestamp('2018-03-22 17:00:00'), 0.061958000000000006, 0.061336,
        0.06194

In [177]:
closes = data_handler.get_latest_bars_values('ETH-BTC', 'close', N=100)

In [180]:
%timeit closes[-1]

158 ns ± 1.37 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [182]:
%timeit data_handler.get_latest_bar_value('ETH-BTC', 'close')

381 ns ± 38 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
data_handler.get_latest_bar_value('ETH-BTC', 'close')

0.075363

In [8]:
data_handler.get_latest_bar_value('ETH-BTC').close

TypeError: get_latest_bar_value() missing 1 required positional argument: 'value_type'